In [ ]:
# 라이브러리 호출
import numpy as np
import pandas as pd

import FinanceDataReader as fdr
from pandas_datareader import data

from datetime import datetime
from dateutil.relativedelta import relativedelta #상대적 날짜 구하기

import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py 
import cufflinks as cf 
cf.go_offline(connected=True)


In [ ]:
### 설정값 입력 ###

# 백테스트 날짜 설정
start_day = '1971-01-01' 
end_day = datetime.today().strftime('%Y-%m-%d')

# 포트폴리오 갯수
portfolio = 3

# 포트폴리오 세부 설정
port1_asset = ['^IXIC', 'VUSTX']
port1_ratio = [1, 1]
port1_leverage = [1, 1]

port2_asset = ['^IXIC', 'VUSTX']
port2_ratio = [6, 4]
port2_leverage = [1, 1]

port3_asset = ['^IXIC', '^IXIC']
port3_ratio = [6, 4]
port3_leverage = [1, -1]

benchmark = ['^IXIC', 'NASDAQ'] #[Ticker, 이름]

# 최초 평가액
initial_balance = 1000

# 리밸런싱 주기(월)
rebalancing_interval = 3

In [ ]:
# 첫 거래일 계산
for port_num in range(1,portfolio+1):
    for i in range(len(locals()['port'+str(port_num)+'_asset'])):
        if datetime.strptime(start_day, '%Y-%m-%d') <= data.get_data_yahoo(locals()['port'+str(port_num)+'_asset'][i], start_day, end_day).index[0]:
            start_day = data.get_data_yahoo(locals()['port'+str(port_num)+'_asset'][i], start_day, end_day).index[0].strftime('%Y-%m-%d')
print('첫 거래일 :', start_day)

# 지수 데이터 생성
for port_num in range(1,portfolio+1):
    for i in range(len(locals()['port'+str(port_num)+'_asset'])):
        locals()['port'+str(port_num)+'_asset'+str(i)+'_data'] = data.get_data_yahoo(locals()['port'+str(port_num)+'_asset'][i], start_day, end_day)

benchmark_data = data.get_data_yahoo(benchmark[0], start_day, end_day)
        
# 날짜 리스트 생성
date = benchmark_data.index

time_period = (date[-1] - date[0]).days/365 # 거래 기간 계산

# 가격, 등락률 데이터 생성
for port_num in range(1,portfolio+1):
    for i in range(len(locals()['port'+str(port_num)+'_asset'])):
        locals()['port'+str(port_num)+'_asset'+str(i)+'_price'] = locals()['port'+str(port_num)+'_asset'+str(i)+'_data']['Adj Close'].to_list()
        locals()['port'+str(port_num)+'_asset'+str(i)+'_change'] = np.multiply(locals()['port'+str(port_num)+'_asset'+str(i)+'_data']['Adj Close'].pct_change().to_list(),[100 * locals()['port'+str(port_num)+'_leverage'][i]])
    
benchmark_price = benchmark_data['Adj Close'].to_list()
benchmark_change = np.multiply(benchmark_data['Adj Close'].pct_change().to_list(),[100])

In [ ]:
# 백테스트
def backtest(port_num):
    
    # 첫 거래일
    for i in range(len(globals()['port'+str(port_num)+'_asset'])):
        locals()['asset'+str(i)+'_balance'] = [initial_balance * globals()['port'+str(port_num)+'_ratio'][i]/sum(globals()['port'+str(port_num)+'_ratio'])]
        
    total_balance = [initial_balance]
    next_rebalancing = [date[0] + relativedelta(months=rebalancing_interval)]
    
    # 이후 거래일 
    for i in range(1, len(date)):
        total_balance_tmp = 0
        
        if date[i] >= next_rebalancing[i-1]: #리밸런싱 하는 날일 때
            next_rebalancing.append(next_rebalancing[i-1] + relativedelta(months=rebalancing_interval))
            for j in range(len(globals()['port'+str(port_num)+'_asset'])):
                locals()['asset'+str(j)+'_balance'].append(total_balance[i-1] * globals()['port'+str(port_num)+'_ratio'][j]/sum(globals()['port'+str(port_num)+'_ratio']) * (1 + globals()['port'+str(port_num)+'_asset'+str(j)+'_change'][i]/100))
                total_balance_tmp = total_balance_tmp + locals()['asset'+str(j)+'_balance'][i]
        else:
            next_rebalancing.append(next_rebalancing[i-1])
            for j in range(len(globals()['port'+str(port_num)+'_asset'])):
                try:
                    locals()['asset'+str(j)+'_balance'].append(locals()['asset'+str(j)+'_balance'][i-1] * (1 + globals()['port'+str(port_num)+'_asset'+str(j)+'_change'][i]/100))
                except:
                    locals()['asset'+str(j)+'_balance'].append(locals()['asset'+str(j)+'_balance'][i-1])
                total_balance_tmp = total_balance_tmp + locals()['asset'+str(j)+'_balance'][i]
                                         
        total_balance.append(total_balance_tmp)
        
    backtest_df = pd.DataFrame()
    backtest_df['date'] = date
    backtest_df.set_index('date', inplace=True)
    backtest_df['total'] = total_balance
    for i in range(len(globals()['port'+str(port_num)+'_asset'])):
        backtest_df['asset'+str(i)] = locals()['asset'+str(i)+'_balance']
    
#     backtest_df['next_rebalancing'] = next_rebalancing
    print('portfolio'+str(port_num)+' 테스트 완료')                                     
    
    return(backtest_df)

In [ ]:
def benchmark_test():
    benchmark_balance = [initial_balance]
    for i in range(1, len(date)):
        benchmark_balance.append(benchmark_balance[i-1] * (1 + benchmark_change[i]/100))
    
    benchmark_df = pd.DataFrame()
    benchmark_df['date'] = date
    benchmark_df.set_index('date', inplace=True)
    benchmark_df['benchmark'] = benchmark_balance
    
    return(benchmark_df)

In [ ]:
# 평가액 기준 성과지표 만들기 함수
def balance_result(balance):
    
    change = [0]
    cum_return = [0]
    prev_high = [0]
    prev_high_tmp = 0
    drawdown = [0]
    MDD = [0]
    MDD_tmp = 0
    CAGR = [0]
    STDEV = [0]
    sharpe = [0]
    
    for i in range(1, len(date)):
        change.append((balance[i]/balance[i-1] - 1)*100)
        
        cum_return.append((balance[i]/balance[0] - 1) * 100)
        
        if prev_high_tmp < cum_return[i]:
            prev_high_tmp = cum_return[i] 
        else: 
            prev_high_tmp = prev_high_tmp
        prev_high.append(prev_high_tmp)
        
        CAGR.append(((balance[i]/balance[0]) ** (1 / float(time_period)) - 1) * 100)
        
        drawdown.append(((cum_return[i] + 100)/(prev_high[i] + 100) - 1) * 100)

        if MDD_tmp > drawdown[i]:
            MDD_tmp = drawdown[i]
        else:
            MDD_tmp = MDD_tmp
        MDD.append(MDD_tmp)

        STDEV.append(np.std(change))
        
        if STDEV[i] != 0:
            sharpe.append(np.sqrt(252) * np.mean(change) / np.std(change))
        else:
            sharpe.append(0)
        
    return(change, cum_return, prev_high, CAGR, drawdown, MDD, STDEV, sharpe)

In [ ]:
# 백테스트 결과 정리
def backtest_result(backtest_df):
    backtest_result_df = pd.DataFrame()
    backtest_result_df['date'] = date
    backtest_result_df.set_index('date', inplace=True)
    result_label = ['change', 'cum_return', 'prev_high', 'CAGR', 'drawdown', 'MDD', 'STDEV', 'sharpe']
    for i in backtest_df.columns:
        locals()[i + '_result'] = balance_result(backtest_df[i].to_list())
        backtest_result_df[i + '_balance'] = backtest_df[i].to_list()
        print(i + ' 완료')
    for j in range(len(result_label)):
        for i in backtest_df.columns:
            backtest_result_df[i + '_' + result_label[j]] = locals()[i + '_result'][j]
    return(backtest_result_df)

In [ ]:
# 연도별, 월별 수익률
def annual_monthly_result(backtest_df):
    
    annual_return_df = pd.DataFrame()
    monthly_return_df = pd.DataFrame()
    
    for label in backtest_df.columns:
        return_year = []
        return_month = []
        locals()[label + '_annual_return'] = []
        locals()[label + '_monthly_return'] = []
        locals()[label + '_annual_start_balance'] = []
        locals()[label + '_annual_end_balance'] = []
        locals()[label + '_monthly_start_balance'] = []
        locals()[label + '_monthly_end_balance'] = []
        locals()[label + '_annual_start'] = backtest_df[label].to_list()[0]
        locals()[label + '_monthly_start'] = backtest_df[label].to_list()[0]
        for i in range(1, len(date)):
            if date[i].year != date[i-1].year:
                return_year.append(date[i-1].year)
                locals()[label + '_annual_return'].append((backtest_df[label].to_list()[i-1]/locals()[label + '_annual_start'] - 1) * 100)
                locals()[label + '_annual_start_balance'].append(locals()[label + '_annual_start'])
                locals()[label + '_annual_end_balance'].append(backtest_df[label].to_list()[i-1])
                locals()[label + '_annual_start'] = backtest_df[label].to_list()[i]
            elif date[i] == date[-1]: # 마지막 거래일
                return_year.append(date[i].year)
                locals()[label + '_annual_return'].append((backtest_df[label].to_list()[i]/locals()[label + '_annual_start'] - 1) * 100)
                locals()[label + '_annual_start_balance'].append(locals()[label + '_annual_start'])
                locals()[label + '_annual_end_balance'].append(backtest_df[label].to_list()[i-1])

            if date[i].month != date[i-1].month:
                return_month.append(date[i-1].strftime('%Y-%m'))
                locals()[label + '_monthly_return'].append((backtest_df[label].to_list()[i-1]/locals()[label + '_monthly_start'] - 1) * 100)
                locals()[label + '_monthly_start_balance'].append(locals()[label + '_monthly_start'])
                locals()[label + '_monthly_end_balance'].append(backtest_df[label].to_list()[i-1])
                locals()[label + '_monthly_start'] = backtest_df[label].to_list()[i]
            elif date[i] == date[-1]: # 마지막 거래일
                return_month.append(date[i].strftime('%Y-%m'))
                locals()[label + '_monthly_return'].append((backtest_df[label].to_list()[i]/locals()[label + '_monthly_start'] - 1) * 100)
                locals()[label + '_monthly_start_balance'].append(locals()[label + '_monthly_start'])
                locals()[label + '_monthly_end_balance'].append(backtest_df[label].to_list()[i-1])
                
        annual_return_df[label + '_annual_return'] = locals()[label + '_annual_return']
        monthly_return_df[label + '_monthly_return'] = locals()[label + '_monthly_return']
    annual_return_df['return_year'] = return_year
    annual_return_df.set_index('return_year', inplace=True)
    monthly_return_df['return_month'] = return_month
    monthly_return_df.set_index('return_month', inplace=True)
    
    print('연도별, 월별 수익률 생성 완료')
    
    return(annual_return_df, monthly_return_df)

In [ ]:
for port_num in range(1,portfolio+1):
    locals()['port'+str(port_num)+'_backtest_df'] = backtest(port_num)
    locals()['port'+str(port_num)+'_backtest_result_df'] = backtest_result(locals()['port'+str(port_num)+'_backtest_df'])
    locals()['port'+str(port_num)+'_annual_monthly_return'] = annual_monthly_result(locals()['port'+str(port_num)+'_backtest_df'])
    locals()['port'+str(port_num)+'_annual_return_df'] = locals()['port'+str(port_num)+'_annual_monthly_return'][0]
    locals()['port'+str(port_num)+'_monthly_return_df'] = locals()['port'+str(port_num)+'_annual_monthly_return'][1]

benchmark_test_df = benchmark_test()
benchmark_result_df = backtest_result(benchmark_test_df)
benchmark_annual_monthly_return = annual_monthly_result(benchmark_test_df)
benchmark_annual_return_df = benchmark_annual_monthly_return[0]
benchmark_monthly_return_df = benchmark_annual_monthly_return[1]

In [ ]:
# csv 파일 만들기
csv_name = ''
for port_num in range(1,portfolio+1):
    for i in range(len(globals()['port'+str(port_num)+'_asset'])):
        csv_name = csv_name + globals()['port'+str(port_num)+'_asset'][i] + '(' + str(int(globals()['port'+str(port_num)+'_ratio'][i]/sum(globals()['port'+str(port_num)+'_ratio'])*100)) + '%,' + str(globals()['port'+str(port_num)+'_leverage'][i]) + 'x)'
    csv_name = '정적 자산배분_포트폴리오'+ str(port_num) + '_' + csv_name + '-' + str(rebalancing_interval) + '개월 리밸런싱_' + start_day + '~' + end_day + '(' + str(round(time_period,1)) + '년).csv'
    
    try:
        locals()['port'+str(port_num)+'_backtest_result_df'].to_csv(csv_name, encoding='euc-kr')
    except: # 파일 열려있어서 에러 뜨면 건너뛰기
        pass
    
    csv_name = ''

In [ ]:
# 성과지표
portfolio_returns = pd.DataFrame(columns=['Portfolio', 'port_detail', 'Initial Balance', 'Final Balance', 'cum_return', 'CAGR', 'MDD', 'STDEV', 'Sharpe Ratio'])

detail = ''
for port_num in range(1,portfolio+1):
    for i in range(len(globals()['port'+str(port_num)+'_asset'])):
            detail = detail + globals()['port'+str(port_num)+'_asset'][i] + '(' + str(int(globals()['port'+str(port_num)+'_ratio'][i]/sum(globals()['port'+str(port_num)+'_ratio'])*100)) + '%,' + str(globals()['port'+str(port_num)+'_leverage'][i]) + 'x)'
            
    portfolio_returns.loc[port_num] = [
        'Portfolio ' + str(port_num),
        detail,
        int(locals()['port'+str(port_num)+'_backtest_result_df']['total_balance'][0]),
        int(locals()['port'+str(port_num)+'_backtest_result_df']['total_balance'][-1]),
        str(round(locals()['port'+str(port_num)+'_backtest_result_df']['total_cum_return'][-1],1))+'%',
        str(round(locals()['port'+str(port_num)+'_backtest_result_df']['total_CAGR'][-1],2))+'%',
        str(round(locals()['port'+str(port_num)+'_backtest_result_df']['total_MDD'][-1],2))+'%',
        round(locals()['port'+str(port_num)+'_backtest_result_df']['total_STDEV'][-1],2),
        round(locals()['port'+str(port_num)+'_backtest_result_df']['total_sharpe'][-1],2),
    ]
    detail = ''

portfolio_returns.loc[0] = [
    'Benchmark',
    benchmark[1],
    int(benchmark_result_df['benchmark_balance'][0]),
    int(benchmark_result_df['benchmark_balance'][-1]),
    str(round(benchmark_result_df['benchmark_cum_return'][-1],1))+'%',
    str(round(benchmark_result_df['benchmark_CAGR'][-1],2))+'%',
    str(round(benchmark_result_df['benchmark_MDD'][-1],2))+'%',
    round(benchmark_result_df['benchmark_STDEV'][-1],2),
    round(benchmark_result_df['benchmark_sharpe'][-1],2),
]


In [ ]:
# 라인 차트
def line_plotting(returns, title, plot_type):
    fig = go.Figure()
    
    for port_num in range(1,portfolio+1):
        fig.add_trace(go.Scatter(x=date, y=globals()['port'+str(port_num)+'_backtest_result_df'].loc[:,'total_'+ returns], 
                                 name='Portfolio'+str(port_num),line=dict(width=1)))
    fig.add_trace(go.Scatter(x=date, y=benchmark_result_df.loc[:,'benchmark_'+ returns], 
                                 name='Benchmark',line=dict(width=1)))
    # Edit the layout
    fig.update_layout(title={'text': title, 'x':0.5, 'xanchor': 'center','yanchor': 'top'},
                     yaxis=dict(type=plot_type,autorange=True))
    
    fig.show()

# 바 차트
def bar_plotting(returns, title):
    fig = go.Figure()
    
    for port_num in range(1,portfolio+1):
        fig.add_trace(go.Bar(x=globals()['port'+str(port_num)+'_'+ returns + '_return_df'].index, 
                             y=globals()['port'+str(port_num)+'_'+ returns + '_return_df'].loc[:,'total_'+returns+'_return'],
                             name='Portfolio'+str(port_num)))
        
    fig.add_trace(go.Bar(x=globals()['benchmark_' + returns +'_return_df'].index, 
                             y=globals()['benchmark_' + returns +'_return_df'].loc[:,'benchmark_'+returns+'_return'],
                             name='Benchmark'))
    # Edit the layout
    fig.update_layout(title={'text': title, 'x':0.5, 'xanchor': 'center','yanchor': 'top'},
                     yaxis=dict(type='linear',autorange=True))
    
    fig.show()

In [ ]:
# 결과 출력
print('테스트기간 : ', start_day, '~', end_day, '/', round(time_period,1), '년')
portfolio_returns

In [ ]:
#### 차트 출력
line_plotting('cum_return', '누적 수익률 비교', 'linear')
line_plotting('drawdown', '낙폭 비교', 'linear')
line_plotting('balance', '평가액 비교(로그)', 'log')

bar_plotting('annual', '연도별 수익률 비교')
bar_plotting('monthly', '월별 수익률 비교')
